In [3]:
import numpy as np
from sklearn.datasets import make_multilabel_classification, make_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.utils import shuffle


from sklearn.datasets import make_classification
from sklearn.utils import shuffle
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

In [12]:
>>> X, y1 = make_classification(n_samples=10000, n_features=100, n_informative=30, n_classes=3, random_state=1)
>>> y2 = shuffle(y1, random_state=1)
>>> y3 = shuffle(y1, random_state=2)
>>> Y = np.vstack((y1, y2, y3)).T
>>> n_samples, n_features = X.shape # 10,100
>>> n_outputs = Y.shape[1] # 3
>>> n_classes = 3

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=1)

In [23]:
forest = RandomForestClassifier(random_state=1)
multi_target_forest = MultiOutputClassifier(forest, n_jobs=-1)
multi_target_forest.fit(X_train, y_train)
y_pred = multi_target_forest.predict(X_test)

In [24]:
multi_target_forest.score(X_test, y_test)

0.0864

---

In [30]:
mlp get_paramsMLPClassifier(random_state=1, max_iter=300)
multi_target_nn = MultiOutputClassifier(mlp, n_jobs=-1)
multi_target_nn.fit(X_train, y_train)
y_pred = multi_target_nn.predict(X_test)

/usr/local/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


In [29]:
multi_target_nn.score(X_test, y_test)

0.1056

In [31]:
mlp.get_params()

{'activation': 'relu',
 'alpha': 0.0001,
 'batch_size': 'auto',
 'beta_1': 0.9,
 'beta_2': 0.999,
 'early_stopping': False,
 'epsilon': 1e-08,
 'hidden_layer_sizes': (100,),
 'learning_rate': 'constant',
 'learning_rate_init': 0.001,
 'max_fun': 15000,
 'max_iter': 300,
 'momentum': 0.9,
 'n_iter_no_change': 10,
 'nesterovs_momentum': True,
 'power_t': 0.5,
 'random_state': 1,
 'shuffle': True,
 'solver': 'adam',
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': False,
 'warm_start': False}

---

In [9]:
import sys
import re
import numpy as np
import pandas as pd
import optuna
import joblib
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

from sqlalchemy import create_engine
from sklearn.metrics import accuracy_score, recall_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /Users/zacks/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/zacks/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/zacks/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [10]:
def tokenize(message, stem='lemm'):
    """Text processing.
    
    Args:
        stem(str): stem or lemm.
        
    Returns:
        list: Cleaned tokens.
    """
    # 1. Cleaning

    # 2. Normalization
    text = re.sub(r"[^a-zA-Z0-9]", " ", message.lower())

    # 3. Tokenization
    tokens = word_tokenize(text)

    # 4. Stop Word Removal
    stop_words = stopwords.words("english")
    tokens = list(filter(lambda w: w not in stop_words, tokens))

    # 5. Part of Speech Tagging / Named Entity Recognition

    # 6. Stemming or Lemmatization
    # Because the targets are not roots, we should use Lemmatization

    clean_tokens = []
    if stem == 'stem':
        stemmer = PorterStemmer()
        for tok in tokens:
            clean_tok = stemmer.stem(tok).strip()
            clean_tokens.append(clean_tok)
    else:
        lemmatizer = WordNetLemmatizer()
        for tok in tokens:
            clean_tok = lemmatizer.lemmatize(tok).strip()
            clean_tokens.append(clean_tok)

    return clean_tokens

In [11]:
model = joblib.load('model.pkl')

In [19]:
def load_data(data_file):
    # read in file
    engine = create_engine(f'sqlite:///{data_file}.db')
    df = pd.read_sql(f'select * from {data_file}', con=engine)

    # define features and label arrays
    X = df.message
    Y = df.loc[:, 'related':]
    target_names = Y.columns
    y = Y.to_numpy()

    return X, y, target_names

X, y, target_names = load_data('disaster_response')

In [20]:
target_names

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [89]:
%%time
pred = model.predict(['I need food', 'I am hungry'])

CPU times: user 10.4 ms, sys: 1.49 ms, total: 11.9 ms
Wall time: 10.7 ms


In [90]:
df = pd.DataFrame(columns=target_names, data=pred)
df

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [91]:
df[['food']]

,food
0,1
1,1


In [34]:
X

0        Weather update - a cold front from Cuba that c...
1                  Is the Hurricane over or is it not over
2                          Looking for someone but no name
3        UN reports Leogane 80-90 destroyed. Only Hospi...
4        says: west side of Haiti, rest of the country ...
                               ...                        
26211    The training demonstrated how to enhance micro...
26212    A suitable candidate has been selected and OCH...
26213    Proshika, operating in Cox's Bazar municipalit...
26214    Some 2,000 women protesting against the conduc...
26215    A radical shift in thinking came about as a re...
Name: message, Length: 26216, dtype: object

In [36]:
'Weather' in X[0]

True

In [94]:
data_file = 'disaster_response'
engine = create_engine(f'sqlite:///{data_file}.db')
df = pd.read_sql(f'select * from {data_file}', con=engine)

In [79]:
df[df.message.str.contains(r'water')].message

7        Please, we need tents and water. We are in Sil...
10       There's nothing to eat and water, we starving ...
12       I am in Thomassin number 32, in the area named...
16       We need food and water in Klecin 12. We are dy...
22       There's a lack of water in Moleya, please info...
                               ...                        
26127    Officials have opened a barrage on the river n...
26144    They will include basic cooking supplies, plat...
26165    One of the first organizations to respond to t...
26168    * Sanitation, (the importance of clean drinkin...
26197    We're providing clean water to people who woul...
Name: message, Length: 2581, dtype: object

In [95]:
len(df[df.message.str.contains(r'food')].index)

2420

In [96]:
len(df[df.message.str.contains(r'hungry')].index)

167

In [102]:
np.intersect1d(, len(df[df.message.str.contains(r'food')].index.to_list()))

array([], dtype=int64)

In [98]:
df[df.message.str.contains(r'hungry')].index

Int64Index([   16,    23,   118,   119,   130,   151,   174,   196,   233,
              391,
            ...
            15702, 17445, 18272, 19140, 22612, 23443, 23666, 23834, 24645,
            25818],
           dtype='int64', length=167)

In [103]:
l1 = df[df.message.str.contains(r'food')].index.to_list()
l2 = df[df.message.str.contains(r'hungry')].index.to_list()

In [107]:
np.intersect1d(l1, l2)

array([   16,    23,   119,   196,   862,  1040,  1464,  1480,  1576,
        1677,  1684,  1769,  1873,  2070,  2549,  2576,  2790,  2812,
        2819,  2831,  2872,  2907,  2947,  3069,  3079,  3749,  3863,
        3889,  4007,  4374,  4941,  5411,  5413,  5428,  6496,  6708,
        6759,  6785,  7078,  7160,  7749,  7937,  8230,  9889, 19140,
       22612])

In [112]:
df[df.message.str.contains(r'[(hungry)(food)]')].index

Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                9,
            ...
            26206, 26207, 26208, 26209, 26210, 26211, 26212, 26213, 26214,
            26215],
           dtype='int64', length=25637)

In [113]:
len(df)

26216

In [16]:
import nltk

In [17]:
nltk.download?

Signature:
nltk.download(
    info_or_id=None,
    download_dir=None,
    quiet=False,
    force=False,
    prefix='[nltk_data] ',
    halt_on_error=True,
    raise_on_error=False,
    print_error_to=<ipykernel.iostream.OutStream object at 0x10bc318e0>,
)
Docstring: <no docstring>
File:      /usr/local/lib/python3.9/site-packages/nltk/downloader.py
Type:      method


In [20]:
nltk.data.find('tokenizers/punkt')

FileSystemPathPointer('/Users/zacks/nltk_data/tokenizers/punkt/PY3')

In [21]:
nltk.data.find('stem/wordnet')

LookupError: 
**********************************************************************
  Resource [93mwordnet[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('wordnet')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mstem/wordnet[0m

  Searched in:
    - '/Users/zacks/nltk_data'
    - '/usr/local/opt/python@3.9/Frameworks/Python.framework/Versions/3.9/nltk_data'
    - '/usr/local/opt/python@3.9/Frameworks/Python.framework/Versions/3.9/share/nltk_data'
    - '/usr/local/opt/python@3.9/Frameworks/Python.framework/Versions/3.9/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [22]:
nltk.data.find?

Signature: nltk.data.find(resource_name, paths=None)
Docstring:
Find the given resource by searching through the directories and
zip files in paths, where a None or empty string specifies an absolute path.
Returns a corresponding path name.  If the given resource is not
found, raise a ``LookupError``, whose message gives a pointer to
the installation instructions for the NLTK downloader.

Zip File Handling:

  - If ``resource_name`` contains a component with a ``.zip``
    extension, then it is assumed to be a zipfile; and the
    remaining path components are used to look inside the zipfile.

  - If any element of ``nltk.data.path`` has a ``.zip`` extension,
    then it is assumed to be a zipfile.

  - If a given resource name that does not contain any zipfile
    component is not found initially, then ``find()`` will make a
    second attempt to find that resource, by replacing each
    component *p* in the path with *p.zip/p*.  For example, this
    allows ``find()`` to map the reso